In [1]:
# import dependencies
import pandas as pd
from pathlib import Path

# paths for .csv file
countymarket_loc = Path("../Resources/cleaned_county_market_tracker.csv")
cleaned_countymarket_loc = Path("../Resources/fullcleaned_county_market_tracker.csv")

# read .csv file
cleaned_countymarket = pd.read_csv(countymarket_loc)
cleaned_countymarket.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2024-03-01,2024-03-31,30,county,5,1901,f,"Mercer County, NJ",NaN,New Jersey,...,0.088323,0.148607,0.018782,0.020402,0.412587,0.067455,0.028940,"Trenton, NJ",45940.0,2024-05-19 14:23:50
1,2012-09-01,2012-09-30,30,county,5,1896,f,"Cumberland County, NJ",NaN,New Jersey,...,0.000000,NaN,NaN,NaN,1.000000,1.000000,0.000000,"Vineland, NJ",47220.0,2024-05-19 14:23:50
2,2018-02-01,2018-02-28,30,county,5,1909,f,"Sussex County, NJ",NaN,New Jersey,...,0.073804,0.199525,-0.051904,-0.023776,0.266129,0.026813,0.011985,"Newark, NJ",35084.0,2024-05-19 14:23:50
3,2014-07-01,2014-07-31,30,county,5,1898,f,"Gloucester County, NJ",NaN,New Jersey,...,-0.015844,0.269880,0.009255,0.034798,0.327684,0.008636,0.104081,"Camden, NJ",15804.0,2024-05-19 14:23:50
4,2020-10-01,2020-10-31,30,county,5,1896,f,"Cumberland County, NJ",NaN,New Jersey,...,0.142857,0.093750,-0.027462,-0.186250,0.142857,-0.079365,-0.357143,"Vineland, NJ",47220.0,2024-05-19 14:23:50


In [2]:
# checking columns
cleaned_countymarket.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [3]:
# checking data types of price drops columns
print(cleaned_countymarket[['price_drops', 'price_drops_mom']].dtypes)

price_drops        float64
price_drops_mom    float64
dtype: object


In [4]:
# defining which columns need to be converted to %
percent_columns = ['median_sale_price_mom', 'median_list_price_mom', 'median_ppsf_mom', 'median_list_ppsf_mom', 
                   'homes_sold_mom', 'pending_sales_mom', 'new_listings_mom', 'months_of_supply_mom', 
                   'median_dom_mom', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'sold_above_list_mom', 'price_drops', 'price_drops_mom', 'off_market_in_two_weeks', 
                   'off_market_in_two_weeks_mom', 'sold_above_list']

# sort the data by 'region', 'date', and 'property type' in ascending order
sorted_countymarket = cleaned_countymarket.sort_values(by=['region', 'period_begin', 'property_type'])

# cleaning, and keeping the correct columns
reduced_countymarket = sorted_countymarket.loc[:, ['period_begin', 'period_end', 'region', 'state_code', 'property_type', 'median_sale_price',
                                                'median_sale_price_mom', 'median_list_price', 'median_list_price_mom',
                                                'median_ppsf', 'median_ppsf_mom', 'median_list_ppsf', 'median_list_ppsf_mom', 'homes_sold',
                                                'homes_sold_mom', 'pending_sales', 'pending_sales_mom', 'new_listings', 'new_listings_mom',
                                                'months_of_supply', 'months_of_supply_mom', 'median_dom', 'median_dom_mom',
                                                'avg_sale_to_list', 'avg_sale_to_list_mom', 'sold_above_list', 'sold_above_list_mom',
                                                'price_drops', 'price_drops_mom','off_market_in_two_weeks',
                                                'off_market_in_two_weeks_mom']]

# change 'NaN' to '0'
reduced_countymarket[['price_drops', 'price_drops_mom']] = reduced_countymarket[['price_drops', 'price_drops_mom']].fillna(0)

# convert percent columns to percentage format
for column in percent_columns:
    reduced_countymarket[column] = reduced_countymarket[column] * 100
    reduced_countymarket[column] = reduced_countymarket[column].apply(lambda x: f'{x:.2f}%' if not pd.isnull(x) else 'NaN')

# ensure all columns are displayed
pd.set_option('display.max_columns', None)

# display first 15 rows
reduced_countymarket.head()

,period_begin,period_end,region,state_code,property_type,median_sale_price,median_sale_price_mom,median_list_price,median_list_price_mom,median_ppsf,median_ppsf_mom,median_list_ppsf,median_list_ppsf_mom,homes_sold,homes_sold_mom,pending_sales,pending_sales_mom,new_listings,new_listings_mom,months_of_supply,months_of_supply_mom,median_dom,median_dom_mom,avg_sale_to_list,avg_sale_to_list_mom,sold_above_list,sold_above_list_mom,price_drops,price_drops_mom,off_market_in_two_weeks,off_market_in_two_weeks_mom
8399,2012-01-01,2012-01-31,"Atlantic County, NJ",NJ,All Residential,191500.0,8.19%,222450.0,18.96%,112.201389,-5.75%,134.567308,15.42%,170.0,-8.60%,102.0,25.93%,469.0,52.27%,15.7,120.00%,126.0,400.00%,91.32%,0.09%,5.29%,-3.85%,0.00%,0.00%,23.53%,-7.33%
8819,2012-01-01,2012-01-31,"Atlantic County, NJ",NJ,Condo/Co-op,125000.0,-3.85%,222450.0,18.96%,166.204986,-29.92%,134.567308,15.42%,40.0,11.11%,27.0,80.00%,89.0,21.92%,15.7,-270.00%,99.0,-5300.00%,88.52%,-0.22%,2.50%,-0.28%,0.00%,0.00%,18.52%,-8.15%
1506,2012-01-01,2012-01-31,"Atlantic County, NJ",NJ,Multi-Family (2-4 Unit),190000.0,52.00%,222450.0,18.96%,74.393109,13.50%,134.567308,15.42%,5.0,66.67%,3.0,-25.00%,23.0,21.05%,25.0,-1700.00%,59.0,-24100.00%,89.90%,15.40%,0.00%,0.00%,0.00%,0.00%,0.00%,-50.00%
2380,2012-01-01,2012-01-31,"Atlantic County, NJ",NJ,Single Family Residential,199000.0,2.05%,222450.0,18.96%,111.405836,-3.14%,134.567308,15.42%,115.0,-15.44%,61.0,7.02%,311.0,71.82%,14.3,220.00%,128.0,1200.00%,92.32%,-0.35%,6.96%,-4.81%,0.00%,0.00%,31.15%,1.32%
1619,2012-01-01,2012-01-31,"Atlantic County, NJ",NJ,Townhouse,112500.0,-43.47%,222450.0,18.96%,86.329392,-48.14%,134.567308,15.42%,10.0,-9.09%,11.0,120.00%,46.0,31.43%,27.0,250.00%,246.0,16100.00%,92.03%,5.48%,0.00%,0.00%,0.00%,0.00%,0.00%,-40.00%


In [5]:
# list new names for every column
renamed_countymarket = {
    'period_begin': 'Start Date',
    'period_end': 'End Date',
    'region': 'County',
    'state_code': 'State',
    'property_type': 'Property Type',
    'median_sale_price': 'Median Sale Price',
    'median_sale_price_mom': 'Median Sale Price MoM',
    'median_list_price': 'Median List Price',
    'median_list_price_mom': 'Median List Price MoM',
    'median_ppsf': 'Median Price per Sqft',
    'median_ppsf_mom': 'Median Price per Sqft MoM',
    'median_list_ppsf': 'Median List Price per Sqft',
    'median_list_ppsf_mom': 'Median List Price per Sqft MoM',
    'homes_sold': 'Homes Sold',
    'homes_sold_mom': 'Homes Sold MoM',
    'pending_sales': 'Pending Sales',
    'pending_sales_mom': 'Pending Sales MoM',
    'new_listings': 'New Listings',
    'new_listings_mom': 'New Listings MoM',
    'months_of_supply': 'Months of Supply',
    'months_of_supply_mom': 'Months of Supply MoM',
    'median_dom': 'Median Days on Market',
    'median_dom_mom': 'Median Days on Market MoM',
    'avg_sale_to_list': 'Average Sale to List',
    'avg_sale_to_list_mom': 'Average Sale to List MoM',
    'sold_above_list': 'Sold Above List',
    'sold_above_list_mom': 'Sold Above List MoM',
    'price_drops': 'Price Drops',
    'price_drops_mom': 'Price Drops MoM',
    'off_market_in_two_weeks': 'Off Market in Two Weeks',
    'off_market_in_two_weeks_mom': 'Off Market in Two Weeks MoM'
}

# rename the columns using the list above
new_countrymarket_df = reduced_countymarket.rename(columns=renamed_countymarket)

# remove ', NJ' from county, and remove '/Co-op' from property type.
new_countrymarket_df['County'] = new_countrymarket_df['County'].str.replace(', NJ', '')
new_countrymarket_df['Property Type'] = new_countrymarket_df['Property Type'].str.replace('/Co-op', '')

# remove '.0' from homes sold, pending sales, new listings, months of supply, and median days on market
new_countrymarket_df['Homes Sold'] = new_countrymarket_df['Homes Sold'].fillna(0).astype(int).astype(str)
new_countrymarket_df['Pending Sales'] = new_countrymarket_df['Pending Sales'].fillna(0).astype(int).astype(str)
new_countrymarket_df['New Listings'] = new_countrymarket_df['New Listings'].fillna(0).astype(int).astype(str)
new_countrymarket_df['Months of Supply'] = new_countrymarket_df['Months of Supply'].fillna(0).astype(int).astype(str)
new_countrymarket_df['Median Days on Market'] = new_countrymarket_df['Median Days on Market'].fillna(0).astype(int).astype(str)

# column formatting to $
moneyformatting = ['Median Sale Price', 'Median List Price', 'Median Price per Sqft', 'Median List Price per Sqft']

# Apply formatting to columns
for col in moneyformatting:
    new_countrymarket_df[col] = new_countrymarket_df[col].map('${:,.2f}'.format)

new_countrymarket_df.head()

,Start Date,End Date,County,State,Property Type,Median Sale Price,Median Sale Price MoM,Median List Price,Median List Price MoM,Median Price per Sqft,Median Price per Sqft MoM,Median List Price per Sqft,Median List Price per Sqft MoM,Homes Sold,Homes Sold MoM,Pending Sales,Pending Sales MoM,New Listings,New Listings MoM,Months of Supply,Months of Supply MoM,Median Days on Market,Median Days on Market MoM,Average Sale to List,Average Sale to List MoM,Sold Above List,Sold Above List MoM,Price Drops,Price Drops MoM,Off Market in Two Weeks,Off Market in Two Weeks MoM
8399,2012-01-01,2012-01-31,Atlantic County,NJ,All Residential,"$191,500.00",8.19%,"$222,450.00",18.96%,$112.20,-5.75%,$134.57,15.42%,170,-8.60%,102,25.93%,469,52.27%,15,120.00%,126,400.00%,91.32%,0.09%,5.29%,-3.85%,0.00%,0.00%,23.53%,-7.33%
8819,2012-01-01,2012-01-31,Atlantic County,NJ,Condo,"$125,000.00",-3.85%,"$222,450.00",18.96%,$166.20,-29.92%,$134.57,15.42%,40,11.11%,27,80.00%,89,21.92%,15,-270.00%,99,-5300.00%,88.52%,-0.22%,2.50%,-0.28%,0.00%,0.00%,18.52%,-8.15%
1506,2012-01-01,2012-01-31,Atlantic County,NJ,Multi-Family (2-4 Unit),"$190,000.00",52.00%,"$222,450.00",18.96%,$74.39,13.50%,$134.57,15.42%,5,66.67%,3,-25.00%,23,21.05%,25,-1700.00%,59,-24100.00%,89.90%,15.40%,0.00%,0.00%,0.00%,0.00%,0.00%,-50.00%
2380,2012-01-01,2012-01-31,Atlantic County,NJ,Single Family Residential,"$199,000.00",2.05%,"$222,450.00",18.96%,$111.41,-3.14%,$134.57,15.42%,115,-15.44%,61,7.02%,311,71.82%,14,220.00%,128,1200.00%,92.32%,-0.35%,6.96%,-4.81%,0.00%,0.00%,31.15%,1.32%
1619,2012-01-01,2012-01-31,Atlantic County,NJ,Townhouse,"$112,500.00",-43.47%,"$222,450.00",18.96%,$86.33,-48.14%,$134.57,15.42%,10,-9.09%,11,120.00%,46,31.43%,27,250.00%,246,16100.00%,92.03%,5.48%,0.00%,0.00%,0.00%,0.00%,0.00%,-40.00%


In [8]:
print(new_countrymarket_df['County'].nunique())
print(new_countrymarket_df['County'].unique())

21
['Atlantic County' 'Bergen County' 'Burlington County' 'Camden County'
 'Cape May County' 'Cumberland County' 'Essex County' 'Gloucester County'
 'Hudson County' 'Hunterdon County' 'Mercer County' 'Middlesex County'
 'Monmouth County' 'Morris County' 'Ocean County' 'Passaic County'
 'Salem County' 'Somerset County' 'Sussex County' 'Union County'
 'Warren County']


In [57]:
new_countrymarket_df.to_csv(cleaned_countymarket_loc, index=False)